# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [1]:
#| hide
from nbdev.showdoc import *

In [2]:
#| export
def foo(): pass

In [18]:
from torch.cuda.amp import autocast
from torch.nn.parallel import DataParallel
import torch
from torch import nn 
from tqdm.auto import tqdm
from fastcore.all import *
from pathlib import Path
import numpy as np
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2


In [17]:
p_aug = 0.05
class CFG:

    # target
    target_class = number_of_classes = 1


    # Model
    model_name = 'Unet'
    backbone = 'se_resnext50_32x4d'

    in_channel = 1

    ######################
        # training CFG #
    ######################

    img_size = 512
    input_size = 512

    train_batch_size = 16
    valid_batch_size = train_batch_size * 2
    epoch = 2
    lr = 6e-4
    chopping_percentage = 1e-3
    ########################
        # fold #
    ########################
    valid_id = 1

    ######################
        # augmentation #
    ###################### 

    train_aug = A.Compose([
        A.Rotate(limit=270, p=0.5),
        A.RandomScale(
            scale_limit=(0.8,1.25), 
            interpolation=cv2.INTER_CUBIC,
            p=p_aug),
        A.RandomCrop(height=img_size, width=img_size, p=p_aug),
        ToTensorV2(p=1.0, transpose_mask=True),
               ])
    valid_aug = A.Compose([ToTensorV2(p=1.0, transpose_mask=True)])





In [19]:
class CustomModel(nn.Module):
    def __init__(self, CFG, weight=None):
       super().__init__() 
       self.model = smp.Unet(
            encoder_name=CFG.backbone, 
            encoder_weights=weight, 
            in_channels=CFG.in_channel, 
            classes=CFG.number_of_classes,
            activation=None)

    def forward(self, x):
        out = self.model(x)
        return out[:, 0]


In [21]:
def build_model(weights=None):
    model = CustomModel(CFG, weight=weights)
    print(f"model name: {CFG.model_name}")
    print(f"backbone: {CFG.backbone}")
    return model.cuda()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()